In [13]:
import pandas as pd

In [24]:
asistencia_2_parques = pd.read_csv("cvs/attendance.csv")

In [25]:
asistencia_2_parques

,USAGE_DATE,FACILITY_NAME,attendance
0,2018-06-01,PortAventura World,46804
1,2018-06-01,Tivoli Gardens,20420
2,2018-06-02,PortAventura World,57940
3,2018-06-02,Tivoli Gardens,29110
4,2018-06-03,PortAventura World,44365
...,...,...,...
2362,2022-07-24,Tivoli Gardens,26798
2363,2022-07-25,PortAventura World,53764
2364,2022-07-25,Tivoli Gardens,27061
2365,2022-07-26,PortAventura World,26391


In [26]:
asistencia_2_parques["USAGE_DATE"] = pd.to_datetime(asistencia_2_parques["USAGE_DATE"])

In [29]:
asistencia_reciente_2 = asistencia_2_parques.sort_values(by="USAGE_DATE", ascending=False)

In [30]:
asistencia_reciente_2

,USAGE_DATE,FACILITY_NAME,attendance
2366,2022-07-26,Tivoli Gardens,14882
2365,2022-07-26,PortAventura World,26391
2364,2022-07-25,Tivoli Gardens,27061
2363,2022-07-25,PortAventura World,53764
2362,2022-07-24,Tivoli Gardens,26798
...,...,...,...
5,2018-06-03,Tivoli Gardens,23727
2,2018-06-02,PortAventura World,57940
3,2018-06-02,Tivoli Gardens,29110
1,2018-06-01,Tivoli Gardens,20420


In [48]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [49]:
url = "https://statbase.org/datasets/tourism-and-leisure/theme-parks-attendance/?syear=2023&frmreq=0&sproc=proc&filter=&sort="
resp = requests.get(url)
resp.raise_for_status()  # para asegurarnos de que la petición fue exitosa
html = resp.text

In [52]:
soup = BeautifulSoup(html, "html.parser")

# Supongamos que los datos están en un <table>…
table = soup.find("table")
soup = BeautifulSoup(resp.text, "html.parser")

In [55]:
rows = soup.find_all("div", class_="it_row")
all_data = []
for row in rows:
    # dentro de la fila, buscar todas las “celdas”
    cells = row.find_all("div", class_="it_cell")
    row_vals = []
    for c in cells:
        text = c.get_text(strip=True)
        row_vals.append(text)
    all_data.append(row_vals)

# Convertir a dataframe
asistencia_parques = pd.DataFrame(all_data)
print(asistencia_parques.head())

                                                   0         1     2
0                                             Object  share, %  Rank
1  MAGIC KINGDOM THEME PARK AT WALT DISNEY WORLD ...      4.79     1
2      DISNEYLAND PARK AT DISNEYLAND RESORT, ANAHEIM      4.66     2
3                     UNIVERSAL STUDIOS JAPAN, OSAKA      4.33     3
4     TOKYO DISNEYLAND AT TOKYO DISNEY RESORT, TOKYO      4.08     4


In [62]:
asistencia_parques

,0,1,2
0,Object,"share, %",Rank
1,MAGIC KINGDOM THEME PARK AT WALT DISNEY WORLD ...,4.79,1
2,"DISNEYLAND PARK AT DISNEYLAND RESORT, ANAHEIM",4.66,2
3,"UNIVERSAL STUDIOS JAPAN, OSAKA",4.33,3
4,"TOKYO DISNEYLAND AT TOKYO DISNEY RESORT, TOKYO",4.08,4
...,...,...,...
66,"FANTASIALANDIA, SANTIAGO",0.26,63
67,"MUNDO PETAPA, GUATEMALA CITY",0.25,64
68,"THEME PARQUE NACIONAL DEL CAFÉ, QUINDIO",0.23,65
69,"PARQUE DE LA COSTA, BUENOS AIRES",0.17,66


In [59]:
header_cells = soup.find_all("div", class_="it_cell")  # pero esto captura *todas* las celdas, tendremos que filtrar
col_names = []
for cell in header_cells:
    # identificar solo aquellos que tienen el atributo data‑scode relevante y quizá están en la fila de encabezado
    if cell.has_attr("data-scode"):
        text = cell.get_text(strip=True)
        col_names.append(text)

print(col_names)

['Object', 'share, %', 'Rank']
